# Imports

In [1]:
import numpy as np
import pandas as pd
import statsmodels as sm
import sklearn as skl
import math
import matplotlib as mpl

import statsmodels.api as sm

In [2]:
ols?

Object `ols` not found.


# Einlesen und aufbereiten

In [2]:
updated_cols = pd.read_excel("catan_data.xlsx").iloc[0].T
df = pd.read_excel("catan_data.xlsx")
renamedict = dict(zip(df.columns, list(updated_cols)))
df = df.rename(columns = renamedict).iloc[1:]

C:\Users\maxfu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\maxfu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
#numerisch machen
for col in df.columns:
    df[col] = pd.to_numeric(df[col],errors = "ignore")

In [7]:
#variable für direkt am hafen gebaut
df["port"] = 0
df.loc[(df["port_wood"]==1) | (df["port_clay"]==1) | (df["port_sheep"]==1) | 
       (df["port_grain"]==1 )| (df["port_ore"]==1) | (df["port_gen"]==1),"port"] = 1

#variable für in hafennähe gebaut
df["prox_port"] = 0
df.loc[(df["prox_port_wood"]==1) | (df["prox_port_clay"]==1) | (df["prox_port_sheep"]==1) | 
       (df["prox_port_grain"]==1 )| (df["prox_port_ore"]==1) | (df["prox_port_gen"]==1),"prox_port"] = 1

#binärvariablen für spieler
for player in df["player"].unique():
    df[player] = 0
    df.loc[df["player"]==player,player] = 1
    
#binärvariablen für startposition
for position in df["position"].unique():
    df["position_"+position] = 0
    df.loc[df["position"]==position,"position_"+position] = 1
    
#binärvariablen für wüstenlocation
for desert in df["desert"].unique():
    df["desert_"+desert] = 0
    df.loc[df["desert"]==desert,"desert_"+desert] = 1
    
#siegervariable erstellen
df["victor"] = 0
df.loc[df["place"]==1,"victor"] = 1
    
#shares berechnen
resources = []
for colname in [col for col in df if "t_sum" in col]:
    resources.append(colname[6:])
for resource in resources:
    df["share_"+resource] = df["p_sum_"+resource] / df["t_sum_"+resource]
    
    
#signifikanter produktionsvorteil
df["advantage"] = 0
for game in df["game"].unique():
    tempdf = df.loc[df["game"]==game]
    tempdf.loc[tempdf["sum"] - tempdf["sum"].mean() > 2,"advantage"] = 1
    df.loc[df["game"]==game] = tempdf.copy()
    
#standardabweichung bei rohstoffen
df["std_resources"] = df[['p_sum_wood','p_sum_clay','p_sum_sheep',
                          'p_sum_grain','p_sum_ore',]].apply("std",axis = 1)

#produktion: split in rohstoff und handelswaren
df["sum_resources"] = df[['p_sum_wood','p_sum_clay','p_sum_sheep',
                          'p_sum_grain','p_sum_ore',]].apply("sum",axis = 1)
df["sum_tradegoods"] = df[["p_sum_paper","p_sum_coin","p_sum_fabric"]].apply("sum",axis = 1)


In [6]:
[col for col in df]

['season',
 'Session',
 'game',
 'loc',
 'player',
 'score',
 'special',
 'place',
 'position',
 'desert',
 'sum',
 'count_num',
 'count_res',
 'p_sum_wood',
 'p_sum_clay',
 'p_sum_sheep',
 'p_sum_grain',
 'p_sum_ore',
 'p_sum_paper',
 'p_sum_coin',
 'p_sum_fabric',
 'port_wood',
 'port_clay',
 'port_sheep',
 'port_grain',
 'port_ore',
 'port_gen',
 'prox_port_wood',
 'prox_port_clay',
 'prox_port_sheep',
 'prox_port_grain',
 'prox_port_ore',
 'prox_port_gen',
 't_sum_wood',
 't_sum_clay',
 't_sum_sheep',
 't_sum_grain',
 't_sum_ore',
 't_sum_paper',
 't_sum_coin',
 't_sum_fabric',
 'port',
 'prox_port',
 'PF',
 'MF',
 'JHC',
 'position_first',
 'position_second',
 'position_third',
 'desert_center',
 'desert_outer',
 'desert_second',
 'share_wood',
 'share_clay',
 'share_sheep',
 'share_grain',
 'share_ore',
 'share_paper',
 'share_coin',
 'share_fabric']

# Auswertungen

## xxv001 - resource share by player

In [44]:
shares = {}
for player in df["player"].unique():
    x = []
    for resource in resources:
        t = df.loc[df[player]==1]["share_"+resource].describe()[["mean","std"]]
        t = pd.DataFrame(t).T.rename(columns = {"mean":"mean_"+player,"std":"std_"+player})
        x.append(t)
    shares[player] = pd.concat(x)

shares = pd.concat(shares.values(),axis = 1)

shares.to_excel("xxv001 - resource shares by player.xlsx")

## xxv002 - regression shares

In [99]:
X = df[[
     'share_wood',"share_clay",'share_grain','share_ore',"share_sheep",
     'share_paper','share_coin','share_fabric',]]

y = df["score"]

#X = sm.add_constant(X)

model = sm.OLS(y, X,missing = "drop",hasconst=None).fit()

model.summary()

## xxv003 - startspieler abhängig von wüstenposition

In [113]:
df.loc[(df["desert_outer"]==1)].groupby("position").agg("mean")

,season,game,score,special,place,sum,count_num,count_res,p_sum_wood,p_sum_clay,...,desert_outer,desert_second,share_wood,share_clay,share_sheep,share_grain,share_ore,share_paper,share_coin,share_fabric
position,,,,,,,,,,,,,,,,,,,,,
first,2022.0,12.411765,8.294118,2.727273,2.294118,30.411765,5.294118,4.705882,4.882353,5.823529,...,1.0,0.0,0.342399,0.415304,0.311644,0.338044,0.326753,0.294102,0.380952,0.340123
second,2022.0,12.411765,9.941176,2.818182,1.941176,31.705882,5.352941,4.235294,4.470588,4.117647,...,1.0,0.0,0.264301,0.319677,0.342055,0.348615,0.391101,0.202240,0.411172,0.274943
third,2022.0,12.411765,10.470588,3.727273,1.764706,32.882353,5.117647,4.294118,5.764706,3.941176,...,1.0,0.0,0.393299,0.265019,0.346300,0.313341,0.282146,0.503658,0.207875,0.384934


## xxv004 - 

In [119]:
X = df[["p_sum_wood","p_sum_clay","p_sum_grain","p_sum_ore","p_sum_sheep"]]

y = df["score"]

In [120]:
model = sm.OLS(y, X,missing = "drop",hasconst=None).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  score   R-squared (uncentered):                   0.909
Model:                            OLS   Adj. R-squared (uncentered):              0.902
Method:                 Least Squares   F-statistic:                              139.0
Date:                Wed, 28 Dec 2022   Prob (F-statistic):                    6.34e-35
Time:                        20:44:46   Log-Likelihood:                         -190.45
No. Observations:                  75   AIC:                                      390.9
Df Residuals:                      70   BIC:                                      402.5
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
p_sum_wood      0.7052      0.091      7.755      0.000       0.524       0.887
p_sum_clay      0.0826      0.089      0.923      0.359      -0.096       0.261
p_sum_grain     0.2602      0.087      3.006      0.004       0.088       0.433
p_sum_ore       0.6097      0.129      4.728      0.000       0.353       0.867
p_sum_sheep     0.3924      0.117      3.365      0.001       0.160       0.625
==============================================================================
Omnibus:                        7.885   Durbin-Watson:                   2.237
Prob(Omnibus):                  0.019   Jarque-Bera (JB):                2.864
Skew:                           0.007   Prob(JB):                        0.239
Kurtosis:                       2.043   Cond. No.                         4.38
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## ports

In [133]:
x = []
for resource in resources:
    if resource not in ["paper","coin","fabric"]:
        tdf = df.loc[df["port_"+resource]==1].copy()
        tres = tdf[["p_sum_"+resource,"score","place"]].copy().rename(columns = {"p_sum_"+resource:"production"})
        tres["resource"] = resource
        x.append(tres)
    
port_res = pd.concat(x)
display(port_res)


#X = port_res[["production"]]
#X = sm.add_constant(X)
#y = port_res["score"]
#model = sm.OLS(y, X,missing = "drop",hasconst=True).fit()
#model.summary()

,production,score,place,resource
33,11,9,2,wood
57,12,11,3,wood
8,6,4,2,clay
55,5,14,1,clay
75,16,7,3,clay
15,7,9,2,sheep
9,14,13,1,grain
30,10,14,1,grain
24,3,5,3,ore
29,3,8,3,ore


In [132]:
x = []
for resource in resources:
    if resource not in ["paper","coin","fabric"]:
        tdf = df.loc[df["prox_port_"+resource]==1].copy()
        tres = tdf[["p_sum_"+resource,"score","place"]].copy().rename(columns = {"p_sum_"+resource:"production"})
        tres["resource"] = resource
        x.append(tres)
    
prox_port_res = pd.concat(x)
display(prox_port_res)


#X = prox_port_res[["production"]]
#X = sm.add_constant(X)
#y = prox_port_res["score"]
#model = sm.OLS(y, X,missing = "drop",hasconst=True).fit()
#model.summary()

,production,score,place,resource
2,10,14,1,wood
3,2,5,3,wood
4,3,7,2,wood
12,5,13,1,wood
14,0,13,1,wood
17,15,13,1,wood
34,4,5,3,wood
37,8,13,1,wood
47,7,7,3,wood
66,8,13,1,wood


In [143]:
prox_port_res.loc[prox_port_res["production"]>6].groupby("resource").agg("mean")

,production,score,place,victor
resource,,,,
clay,8.000000,13.500000,1.00,1.000000
grain,8.500000,6.750000,2.50,0.000000
ore,7.333333,10.666667,2.00,0.333333
sheep,9.000000,10.250000,1.75,0.250000
wood,9.600000,12.000000,1.40,0.800000


In [ ]:
#platzierung nach spieler
for player in df["player"].unique():
    print(player)
    print(df.loc[df[player]==1]["place"].value_counts().sort_index())

In [96]:
#sieg platzierung nach spieler und location

for location in df["loc"].unique():
    tempdf = df.loc[df["loc"]==location]
    for player in df["player"].unique():
        print(location)
        print(player)
        print(tempdf.loc[tempdf[player]==1]["place"].value_counts(normalize = True).sort_index())
        

Husum (Memeler Str.)
PF
1    0.428571
2    0.357143
3    0.214286
Name: place, dtype: float64
Husum (Memeler Str.)
MF
1    0.214286
2    0.357143
3    0.428571
Name: place, dtype: float64
Husum (Memeler Str.)
JHC
1    0.357143
2    0.285714
3    0.357143
Name: place, dtype: float64
Hamburg (Kegelhofstr.)
PF
1    0.538462
2    0.307692
3    0.153846
Name: place, dtype: float64
Hamburg (Kegelhofstr.)
MF
1    0.076923
2    0.307692
3    0.615385
Name: place, dtype: float64
Hamburg (Kegelhofstr.)
JHC
1    0.384615
2    0.384615
3    0.230769
Name: place, dtype: float64
Online
PF
1    0.666667
3    0.333333
Name: place, dtype: float64
Online
MF
2    0.333333
3    0.666667
Name: place, dtype: float64
Online
JHC
1    0.333333
2    0.666667
Name: place, dtype: float64


PF
1    15
2     9
3     6
Name: place, dtype: int64
MF
1     4
2    10
3    16
Name: place, dtype: int64
JHC
1    11
2    11
3     8
Name: place, dtype: int64


In [109]:
#relativ ausgeglichene rohstoffwahl: effekt auf siegquote
df.loc[(df["std_resources"]>2) & (df["std_resources"]<3)]["victor"].mean()

0.42857142857142855

In [33]:
#
a = df.loc[df["advantage"] ==1].groupby("sum").agg("mean")["score"]
b = df.loc[df["advantage"] ==1].groupby("sum").agg("count")["score"]
pd.concat([a,b],axis = 1)

,score,score
sum,,
31,12.0,2
32,13.5,2
33,10.0,1
34,11.0,5
35,8.0,1
36,9.0,7


In [14]:
#
a = df.groupby("sum").agg("mean")["score"].rename("mean")
b = df.groupby("sum").agg("count")["score"].rename("count")

pd.concat([a,b],axis = 1)

,mean,count
sum,,
0,9.000000,3
24,12.000000,1
25,11.000000,1
26,4.000000,3
27,7.000000,2
28,9.600000,5
29,9.833333,6
30,10.294118,17
31,9.900000,10
